In [5]:
import os
import numpy as np
import xml.etree.ElementTree as ET
from tqdm.notebook import tqdm
import random

from xmlMaker import init_xml
from xmlMaker import add_object
from xmlMaker import prettyXml

# make XML file

In [3]:
xmlFolderPath = os.path.join('../', 'Annotations/')
if not os.path.exists(xmlFolderPath):
    os.mkdir(xmlFolderPath)

folderList = ['valid', 'train', 'test']
for folderName in tqdm(folderList):
    folderPath = '../{}/'.format(folderName)
    annotationsPath = os.path.join(folderPath,'_annotations.csv')
    with open(annotationsPath, 'r') as f:
        csvRead = [x.strip() for x in f.readlines()]

    csvRead = csvRead[1:]
    for line in tqdm(csvRead):
        imageName = line.split('.jpg,')[0]
        xmlPath = os.path.join(xmlFolderPath, '{}.xml'.format(imageName))
        bbox = line.split('smoke,')[-1]
        bbox = [int(x) for x in bbox.split(',')]

        # init xml
        if not os.path.exists(xmlPath):
            init_xml(xmlPath, 'WildFire-Smoke-Dataset-Tensorflow', imageName, 
                    'WildFire-Smoke-Dataset-Tensorflow_smoke_detection', 
                    imageWidth=640, imageHeight=480)

        # add object info
        tree = ET.parse(xmlPath)
        add_object(tree, bbox, 'smoke')
        # reshape xml
        root = tree.getroot()
        prettyXml(root, '\t', '\n')
        # save
        tree.write(xmlPath)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/147 [00:00<?, ?it/s]

  0%|          | 0/516 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

# generate ImageSets and JPEGImages floders

## make JPEGImages

copy images into JPEGImages foldes

## make ImageSets

loading all image names, distribute them into trainset(70%) and testset(3

In [6]:
annotationsPath = '../Annotations/'

allImage = [x.split('.xml')[0] for x in os.listdir(annotationsPath)]

imageSetsPath = '../ImageSets/Main/'
# make ImageSetsPath folder
if not os.path.exists(imageSetsPath):
    os.makedirs(imageSetsPath)
    
allImage = np.array(allImage)

testSize = int(len(allImage) * 0.3)
# shuffle
np.random.shuffle(allImage)
# sample
testIndex = random.sample(range(len(allImage)), testSize)
testList = allImage[np.array(testIndex)]
trainList = [item for item in allImage if item not in testList]
# save
trainTxtPath = os.path.join(imageSetsPath, 'trainval.txt')
testTxtPath = os.path.join(imageSetsPath, 'test.txt')

with open(trainTxtPath, 'w') as f:
    for image in trainList:
        f.write('{}\n'.format(image))

with open(testTxtPath, 'w') as f:
    for image in testList:
        f.write('{}\n'.format(image))